In [ ]:
# !pip install svgpathtools

In [ ]:
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc, disvg, wsvg
import numpy as np

seg1 = QuadraticBezier(100+100j, 200+500j, 300+100j)  # A cubic beginning at (300, 100) and ending at (200, 300)
seg2 = QuadraticBezier(400+150j, 0+100j, 400+200j)  # A line beginning at (200, 300) and ending at (250, 350)
seg3 = Line(50+100j,400+200j)
#Four Intersections
#seg1 = QuadraticBezier(250+100j, 200+500j, 300+100j)  # A cubic beginning at (300, 100) and ending at (200, 300)
#seg2 = QuadraticBezier(400+150j, 0+100j, 400+200j)  # A line beginning at (200, 300) and ending at (250, 350)
path = Path(seg1, seg2, seg3)

In [ ]:
def quadratic_intersection(seg1, seg2):
    """This approach determines intersections by solving the parametric equations
    for Cartesian coordinates associated with each Bezier curve. It simplifies
    the four Cartesian equations into a single fourth-degree polynomial.
    The notation used is as follows:
        'xn' and 'yn' represents coordinates of each control points 0, 1, and 2.
        Parametric forms of coordinates used: X = At^2 + Bt + x0 and Y = Ct^2 + Dt + y0.
        Equation at^4 + bt^3 + ct^2 + d = 0 identifies intersections as roots."""

    if isinstance(seg1,QuadraticBezier) and isinstance(seg2,QuadraticBezier):
        #obtaining cartesian coordinates of control points for each curve
        x01, x11, x21 = seg1.start.real, seg1.control.real, seg1.end.real
        x02, x12, x22 = seg2.start.real, seg2.control.real, seg2.end.real
        y01, y11, y21 = seg1.start.imag, seg1.control.imag, seg1.end.imag
        y02, y12, y22 = seg2.start.imag, seg2.control.imag, seg2.end.imag

        #seting indexes for the quadratic form of a bezzier curve (A,B for x and C,D for y)
        A1, B1 = x01 - 2*x11 + x21, 2*x11 - 2*x01
        A2, B2 = x02 - 2*x12 + x22, 2*x12 - 2*x02
        C1, D1 = y01 - 2*y11 + y21, 2*y11 - 2*y01
        C2, D2 = y02 - 2*y12 + y22, 2*y12 - 2*y02

        #solving equation system {X1=X2, Y1=Y2} for t1
        a = 4*(A1**2*C2**2 - 2*A1*A2*C1*C2 + A2**2*C1**2)*A2**2
        b = -8*(A1*C2 - A2*C1)*A2**2*(A2*D1 - B1*C2)
        c = -4*(A1*(A2*(2*C2*(y01 - y02) + D2**2) - (B2*D2 + 2*C2*(x01 - x02))*C2) - A2**2*(2*C1*(y01 - y02) + D1**2) + A2*(2*B1*C2*D1 - (B2*D2 - 2*C2*(x01 - x02))*C1) - (B1**2*C2 - B2**2*C1)*C2)*A2**2
        d = 4*A2**2*(2*A2**2*D1*(y01 - y02) - A2*(B1*(2*C2*(y01 - y02) + D2**2) - (B2*D2 - 2*C2*(x01 - x02))*D1) + (B1*(B2*D2 + 2*C2*(x01 - x02)) - B2**2*D1)*C2)
        e = 4*A2**2*(A2**2*(y01**2 - 2*y01*y02 + y02**2) + A2*(B2*D2*(y01 - y02) - (2*C2*(y01 - y02) + D2**2)*(x01 - x02)) - (B2**2*(y01 - y02) - B2*D2*(x01 - x02) - C2*(x01**2 - 2*x01*x02 + x02**2))*C2)

        roots = np.roots([a,b,c,d,e])
        t1 = [root for root in roots if root.imag==0]# only real solutions

        #intersection points
        intersections = [seg1.point(t) for t in t1 if t<=1 and t>=0]

        t2 = []
        #t2 as solutions for A1·t1ˆ2 + B1·t1 + x01 = A2·t2ˆ2 + B2·t2 + x02
        for t in t1:
            #A2*t2**2 + B2*t2 + x02 - (A1*t1**2 + B1*t1 + x01)
            t2.extend(np.roots([A2,B2,x02 - (A1*t**2 + B1*t + x01)])) # two solutions for each t1

        #keeping only one solution for each t1
        t2 = [t for t in t2 if any(abs(seg2.point(t) - intersection) < 10e-10 for intersection in intersections)]

        return (list(zip(t1,t2)),intersections)

    else: print("not quadratic")


intersections_list, intersections = quadratic_intersection(seg1, seg2)
t1 = [t[0] for t in intersections_list]
t2 = [t[1] for t in intersections_list]

intersections2 = [seg2.point(t) for t in t2]
print("Parametric t1 and t2:",t1,t2,"Intersections:",intersections,sep='\n')

points = [seg1.start,seg1.control,seg1.end,
            seg2.start,seg2.control,seg2.end]



nodes = points + intersections + intersections2
node_colors = 'r'*len(points) + 'b'*len(intersections) + 'w'*len(intersections2)
line_points = [seg3.point(t[1]) for t in seg1.intersect(seg3)]
nodes.extend(line_points)
print(f"Line/bez intersect: {line_points}")

wsvg(path,nodes=nodes,filename='display_c.svg')
# if any(abs(root) <= 1 and root.imag == 0 for root in roots):
#     disvg(path,nodes=nodes)

In [ ]:
import svgpathtools
print(f"""
{hasattr(svgpathtools, 'quadratic_intersection')}
{hasattr(svgpathtools, 'bezier_intersections')}
""")

In [ ]:
/print("""
4*(a1^(2)*c2^(2)-2*a1*a2*c1*c2+a2^(2)*c1^(2))*a2^(2),-8*(a1*c2-a2*c1)*a2^(2)*(a2*d1-b1*c2),-4*(a1*(a2*(2*c2*(y01-y02)+d2^(2))-(b2*d2+2*c2*(x01-x02))*c2)-a2^(2)*(2*c1*(y01-y02)+d1^(2))+a2*(2*b1*c2*d1-(b2*d2-2*c2*(x01-x02))*c1)-(b1^(2)*c2-b2^(2)*c1)*c2)*a2^(2),4*a2^(2)*(2*a2^(2)*d1*(y01-y02)-a2*(b1*(2*c2*(y01-y02)+d2^(2))-(b2*d2-2*c2*(x01-x02))*d1)+(b1*(b2*d2+2*c2*(x01-x02))-b2^(2)*d1)*c2),4*a2^(2)*(a2^(2)*(y01^(2)-2*y01*y02+y02^(2))+a2*(b2*d2*(y01-y02)-(2*c2*(y01-y02)+d2^(2))*(x01-x02))-(b2^(2)*(y01-y02)-b2*d2*(x01-x02)-c2*(x01^(2)-2*x01*x02+x02^(2)))*c2)
""".replace('+',' + ').replace('-',' - ').replace(',','\n'))

In [ ]:
eq = """
4*(a1^(2)*c2^(2) - 2*a1*a2*c1*c2 + a2^(2)*c1^(2))*a2^(2)
-8*(a1*c2 - a2*c1)*a2^(2)*(a2*d1 - b1*c2)
-4*(a1*(a2*(2*c2*(y01 - y02) + d2^(2)) - (b2*d2 + 2*c2*(x01 - x02))*c2) - a2^(2)*(2*c1*(y01 - y02) + d1^(2)) + a2*(2*b1*c2*d1 - (b2*d2 - 2*c2*(x01 - x02))*c1) - (b1^(2)*c2 - b2^(2)*c1)*c2)*a2^(2)
4*a2^(2)*(2*a2^(2)*d1*(y01 - y02) - a2*(b1*(2*c2*(y01 - y02) + d2^(2)) - (b2*d2 - 2*c2*(x01 - x02))*d1) + (b1*(b2*d2 + 2*c2*(x01 - x02)) - b2^(2)*d1)*c2)
4*a2^(2)*(a2^(2)*(y01^(2) - 2*y01*y02 + y02^(2)) + a2*(b2*d2*(y01 - y02) - (2*c2*(y01 - y02) + d2^(2))*(x01 - x02)) - (b2^(2)*(y01 - y02) - b2*d2*(x01 - x02) - c2*(x01^(2) - 2*x01*x02 + x02^(2)))*c2)
""".replace('₀','0').replace('₁','1').replace('₂','2').replace('^','**').replace('o','0')
eq=eq.replace('a','A').replace('b','B').replace('c','C').replace('d','D').replace('(2)','2').replace('(3)','3').replace('(4)','4')
print(eq)

In [ ]:
t1 = ['a','b','c']
t2 = ['1','2','3']
D=list(zip(t1,t2))
print(D)
z1 = [t[0] for t in D]
z2 = [t[1] for t in D]

